## Load Requirements

In [2]:
import numpy as np


## 1.2 SVD for a Noisy Matrix


#### Matrix Generation

In [3]:
# Generate the matrix

N,M = 100,50

# Form 2 random vectors
u=np.random.rand(N,1)
v=np.random.rand(M,1)

# Compute the rank-1 matrix using outer product
R_1 = u @ v.T

# Calculate the Frobenius norm of this matrix
R_f = np.linalg.norm(R_1, 'fro')

noise_variance = 0.01 * R_f

noise = np.random.normal(0, np.sqrt(noise_variance), (N,M)) 

R_ = R_1 + noise

# Display the shape of the matrix

print("The matrix Shape:", R_.shape)


The matrix Shape: (100, 50)


#### SVD Decomposition

In [4]:
# Decompose the matrix using SVD

U1, s1, V1 = np.linalg.svd(R_, full_matrices=False)

print("U Shape:", U1.shape)

print("s Shape:", s1.shape)

print("V Shape:", V1.shape)


U Shape: (100, 50)
s Shape: (50,)
V Shape: (50, 50)


#### Matrix Reconstruction

In [5]:
# Reconstruct the matrix R using only the first singular value and the corresponidng singular vectors

R_reconstructed = s1[0] * np.outer(U1[:,0], V1[0,:])

#### Analysis

In [6]:
# calculate and output the original values of U and V

U_orginal, s_original, V_original = np.linalg.svd(R_1, full_matrices=False)

print("U_original Shape:", U_orginal)

print("s_original Shape:", s_original)

print("V_original Shape:", V_original)

U_original Shape: [[-0.10521974 -0.6346124   0.00112294 ...  0.00206341  0.01331492
   0.7618841 ]
 [-0.13861847  0.20747591 -0.18305618 ... -0.01181888  0.0178605
   0.14105739]
 [-0.15887453  0.06574152 -0.09912153 ... -0.14036694  0.10980459
   0.03090417]
 ...
 [-0.08411381 -0.0355776  -0.1358308  ... -0.06807754  0.07066409
  -0.03715925]
 [-0.16127616 -0.15535323  0.22973697 ... -0.0978399  -0.13040718
  -0.16290188]
 [-0.13713326  0.09620068  0.11712698 ...  0.08462935  0.07744235
   0.0911559 ]]
s_original Shape: [2.37727461e+01 2.37478589e-15 2.37478589e-15 2.37478589e-15
 2.37478589e-15 2.37478589e-15 2.37478589e-15 2.37478589e-15
 2.37478589e-15 2.37478589e-15 2.37478589e-15 2.37478589e-15
 2.37478589e-15 2.37478589e-15 2.37478589e-15 2.37478589e-15
 2.37478589e-15 2.37478589e-15 2.37478589e-15 2.37478589e-15
 2.37478589e-15 2.37478589e-15 2.37478589e-15 2.37478589e-15
 2.37478589e-15 2.37478589e-15 2.37478589e-15 2.37478589e-15
 2.37478589e-15 2.37478589e-15 2.37478589e-15 

In [7]:
# calculate and output the Reconstructed values of U and V

U_reconstructed, s_constructed, V_reconstructed = np.linalg.svd(R_reconstructed, full_matrices=False)

print("Reconstructed U:", U_reconstructed)

print("Reconstructed V:", V_reconstructed)

print("Reconstructed s:", s_constructed)

Reconstructed U: [[-0.08164613 -0.23046765 -0.01585294 ...  0.00649749 -0.00430998
  -0.96280773]
 [-0.13862972 -0.00972843 -0.09965636 ... -0.15102103 -0.00384195
   0.00458761]
 [-0.17395338  0.25801721 -0.04398593 ...  0.05712565  0.05736543
  -0.02172936]
 ...
 [-0.07476407  0.10726693  0.16526031 ...  0.16723243 -0.16757081
  -0.01971296]
 [-0.17141457  0.1180482  -0.21333445 ... -0.0077754   0.01646208
  -0.02464278]
 [-0.12932091 -0.25386684  0.06898104 ...  0.05666424  0.03432887
   0.05346932]]
Reconstructed V: [[-0.0437993  -0.03562296 -0.23843968 ... -0.09646322 -0.06064143
  -0.03270877]
 [ 0.          0.06825133  0.0081277  ...  0.15759914  0.07625271
  -0.0072758 ]
 [ 0.          0.0627998   0.20669141 ... -0.0167356   0.07997076
   0.00180013]
 ...
 [ 0.         -0.36404679 -0.02948132 ... -0.04156682  0.02617457
   0.31688158]
 [ 0.         -0.15966095 -0.00180538 ...  0.03163339 -0.07859181
  -0.03873101]
 [-0.99904035  0.00156176  0.01045352 ...  0.00422908  0.0026586

In [8]:
# Computer the Root Mean Squared Error between the original matrix R and the reconstructed matrix R_reconstructed 

RMSE = np.sqrt(np.mean((R_1 - R_reconstructed)**2))

print("RMSE:", RMSE)

# Compute the Root Mean Squared Errors of U and V 
# From the value of U_original, V_original and U_reconstructed, V_reconstructed, there is no need to adjust the sign of the singular vectors when calculating the RMSE

RMSE_U = np.sqrt(np.mean((U_orginal - U_reconstructed)**2))

RMSE_V = np.sqrt(np.mean((V_original- V_reconstructed)**2))

print("RMSE_U:", RMSE_U)
print("RMSE_V:", RMSE_V)

RMSE: 0.07999243907684757
RMSE_U: 0.14167936195588504
RMSE_V: 0.19499051993250902


**The impact of noise:**

1. Effect on the Matrix Reconstruction: The RMSE for the reconstructed matrix is pretty small, indicating that the reconstruction closely approximates the original matrix despite the added noise. This is potentially because the reconstruction uses only the first singular value and the corresponding singular vectors, which contains most information of the matrix. This filters out higher-order singular values, which captures a portion of the noise, reducing the effect of noise on the reconstructed matrix.  
2. Effect on the Singular Vectors: The RMSE for the singular vectors suggests that the noise has introduced some discrepancy between the original and reconstructed singular vectors. Singular vectors associated with significant singular values are typically less affected by noise because they represent more prominent structural components of the matrix. But the loss indicates that noise might alter the direction slightly, resulting the overall discrepancies. 


## 1.3 Matrix Factorization of an Imcomplete Matrix

#### Matrix Generation

In [9]:
# Generate the Matrix 

# Calculate the total number of elements and 30% of that
total_elements = N * M
num_missing = int(0.3 * total_elements)

# Randomly select indices to set as missing
missing_indices = np.unravel_index(
    np.random.choice(total_elements, num_missing, replace=False), (N, M)
)

# Create a copy of the original matrix and set the missing values to nan

R_missing = np.copy(R_1)

R_missing[missing_indices] = np.nan



#### Matrix Factorization

In [10]:
# SGD for matrix factorization

def SGD_factorization(learning_rate, regularization, num_epochs, R_missing, output):

    U_factorized = np.random.rand(N,1)
    V_factorized = np.random.rand(M,1) # initialze the U and V matrices

    for epoch in range(num_epochs):

        for i in range(N):
            for j in range(M):
                
                # Only consider the observed values
                if not np.isnan(R_missing[i,j]):

                    prediction = np.dot(U_factorized[i], V_factorized[j])
                    error = R_missing[i,j] - prediction

                    # Update the U and V matrices
                    U_factorized[i] += learning_rate * (error * V_factorized[j] - regularization * U_factorized[i])
                    V_factorized[j] += learning_rate * (error * U_factorized[i] - regularization * V_factorized[j]) 
                    
        if output:
            if epoch % 10 == 0:
            
                # Calculatre the total loss on observed entrices
                observed_indices = ~np.isnan(R_missing)
                loss = np.sum((R_missing[observed_indices] - (U_factorized @ V_factorized.T)[observed_indices])**2)
                print("Epoch:", epoch, "Loss:", loss)
    
    return U_factorized, V_factorized

U_factorized,V_factorized = SGD_factorization(0.01, 0, 200, R_missing,True)

Epoch: 0 Loss: 275.0244727928758
Epoch: 10 Loss: 29.06756786719892
Epoch: 20 Loss: 3.335253914933026
Epoch: 30 Loss: 0.4193239381646615
Epoch: 40 Loss: 0.05669088025885562
Epoch: 50 Loss: 0.008184705309017996
Epoch: 60 Loss: 0.001253506591900327
Epoch: 70 Loss: 0.00020223366775713373
Epoch: 80 Loss: 3.413408977509161e-05
Epoch: 90 Loss: 5.988720286349705e-06
Epoch: 100 Loss: 1.0859074639078807e-06
Epoch: 110 Loss: 2.0249057950748427e-07
Epoch: 120 Loss: 3.866705342451057e-08
Epoch: 130 Loss: 7.534794294973074e-09
Epoch: 140 Loss: 1.4939033548039577e-09
Epoch: 150 Loss: 3.0063230977210334e-10
Epoch: 160 Loss: 6.128137027320058e-11
Epoch: 170 Loss: 1.2631719656974861e-11
Epoch: 180 Loss: 2.6291571824665968e-12
Epoch: 190 Loss: 5.519064792730418e-13


#### Missing Data Imputation

In [11]:
# Use the factorized matrices U and V to reconstruct the matrix R

R_reconstructed_missing = U_factorized @ V_factorized.T



#### Analysis

In [12]:
# Compute the RMSE between the original matrix R and the reconstructed matrix R_reconstructed

RMSE_missing = np.sqrt(np.mean((R_1 - R_reconstructed_missing)**2))

print("RMSE_missing:", RMSE_missing)

RMSE_missing: 7.478288682130052e-09


In [13]:
# Discuss other missing proportions for data generation and discuss their impacts in the reconstruction process

# missing protions from 0.1 to 0.9

missing_proportions = [0.1*i for i in range(1,10)]

missing_proportions.extend([0.95,0.97,0.99])

for missing_proportion in missing_proportions:

    num_missing = int(missing_proportion * total_elements)

    missing_indices = np.unravel_index(
        np.random.choice(total_elements, num_missing, replace=False), (N, M)
    )

    R_missing = np.copy(R_1)
    
    R_missing[missing_indices] = np.nan

    U_factorized,V_factorized = SGD_factorization(0.01, 0,1000, R_missing,False)

    R_reconstructed_missing = U_factorized @ V_factorized.T

    RMSE_missing = np.sqrt(np.mean((R_1 - R_reconstructed_missing)**2))

    print("******* Missing Proportion:", round(missing_proportion,2),"********")
    print("RMSE of recoverd matrix:", RMSE_missing)



******* Missing Proportion: 0.1 ********
RMSE of recoverd matrix: 1.9803661052093288e-15
******* Missing Proportion: 0.2 ********
RMSE of recoverd matrix: 1.5338138003464492e-15


**Discussion impact of the missing value proportion on reconstructed matrix**: 

1. For the low and moderate missing value proportion such as 10% - 70%, the RMSE is very low, indicating that the factorization can approximate the missing value and the observed value accurately.  
2. For the very high missing proportion such as 80% - 99%, the RMSE continues to increase. For the missing value proportion above 90%, the discrepancy of original and reconstructed matrix is notable, indicating the matrix factorization struggles to impute the missing value of the matrix when the matrix is extremely sparse.


In [ ]:
# Generate Matrix with Rank 10

N,M = 100,50

full_rank_matrix = np.random.rand(N, M)
    
# Perform SVD decomposition
U_10, s_10, Vt_10 = np.linalg.svd(full_rank_matrix, full_matrices=False)

# Keep only the top `rank` singular values
s_10[10:] = 0
    
# Reconstruct the matrix

R_10 = U_10 @ np.diag(s_10) @ Vt_10

# choose a moderate missing data proportions
num_missing = int(0.3 * total_elements)

missing_indices = np.unravel_index(
    np.random.choice(total_elements, num_missing, replace=False), (N, M)
)

R_missing_10 = np.copy(R_10)

R_missing_10[missing_indices] = np.nan

# decompose the matrix using different number of singular components

singular_components = [1,2,3,4,5,6,7,8,9,10, 20, 30, 40, 50]

for K in singular_components:

    U_reconstructed, s_constructed, V_reconstructed = np.linalg.svd(R_10, full_matrices=False)
    
    s_constructed[K:] = 0

    R_reconstructed_k = U_reconstructed @ np.diag(s_constructed) @ V_reconstructed

    RMSE_missing = np.sqrt(np.mean((R_10 - R_reconstructed_k)**2))

    print("******* Singular Number:", K ,"********")
    print("RMSE of recovered matrix:", RMSE_missing)
    

******* Singular Number: 1 ********
RMSE_reconstruction: 0.17942279666318597
******* Singular Number: 2 ********
RMSE_reconstruction: 0.1662126903557679
******* Singular Number: 3 ********
RMSE_reconstruction: 0.15369056682766777
******* Singular Number: 4 ********
RMSE_reconstruction: 0.1403840365385495
******* Singular Number: 5 ********
RMSE_reconstruction: 0.12667487566391875
******* Singular Number: 6 ********
RMSE_reconstruction: 0.11196449985081135
******* Singular Number: 7 ********
RMSE_reconstruction: 0.09623198057062363
******* Singular Number: 8 ********
RMSE_reconstruction: 0.0777436333897794
******* Singular Number: 9 ********
RMSE_reconstruction: 0.05461470473860565
******* Singular Number: 10 ********
RMSE_reconstruction: 6.705351340375311e-16
******* Singular Number: 20 ********
RMSE_reconstruction: 6.824197241619624e-16
******* Singular Number: 30 ********
RMSE_reconstruction: 6.939150368551093e-16
******* Singular Number: 40 ********
RMSE_reconstruction: 7.0484321247

**Discussion**:   
I use the a rank-10 matrix with 30% missing values.
1. For the number of singular components smaller than or equal to the Rank of the matrix. As the number of singular components increases, the reconstruction effect increases because for the kth singular compoents (k<=10), the singular components contain part information of the matrix. And the MSE becomes very small as the number of singualr components is equal to the Rank of the matrix because we use nearly all useful information to reconstruct the matrix.  
2. For the number of singular values bigger than the Rank of the matrix. As the number of singular value increases, the reconstruction effect remains stable because singular values after the 10th are quite small so there is little information in those singular components of the matrix. So there is no improvements for reconstruction using k th singular components (k>10).  

## 1.4 Matrix Factorization with Regularization

#### Generate Matrix

In [ ]:
# Calculate the total number of elements and 30% of that
total_elements = N * M

num_missing = int(0.8 * total_elements)

# Randomly select indices to set as missing
missing_indices = np.unravel_index(
    np.random.choice(total_elements, num_missing, replace=False), (N, M)
)

# Create a copy of the original matrix and set the missing values to nan

R_missing = np.copy(R_1)

R_missing[missing_indices] = np.nan

In [ ]:
np.isnan(R_missing).sum()

np.int64(4000)

#### Rank-one factorization using regularization



In [ ]:
# The result of Rank-one factorization using regularization

# set regularization to 0.1 as an example

U_factorized,V_factorized = SGD_factorization(0.01,1e-5, 1000, R_missing,False)

R_reconstructed_missing = U_factorized @ V_factorized.T

RMSE_missing = np.sqrt(np.mean((R_1 - R_reconstructed_missing)**2))

print("R_reconstrcuion", R_reconstructed_missing)

print("R_original",R_1)

print("RMSE of recovered matrix:", RMSE_missing)


R_reconstrcuion [[0.32687131 0.14381702 0.22890687 ... 0.27526085 0.07955473 0.31757722]
 [0.19604535 0.08625614 0.13728989 ... 0.1650913  0.04771399 0.19047109]
 [0.27677472 0.12177549 0.19382439 ... 0.23307412 0.0673621  0.26890504]
 ...
 [0.58900421 0.25915039 0.41247764 ... 0.49600497 0.14335326 0.57225675]
 [0.42631139 0.18756871 0.29854442 ... 0.3590001  0.10375669 0.41418987]
 [0.0439268  0.01932694 0.03076179 ... 0.0369911  0.01069101 0.04267781]]
R_original [[0.32687641 0.14381722 0.22890861 ... 0.27525951 0.07955329 0.31758503]
 [0.19605098 0.0862574  0.13729274 ... 0.16509266 0.04771375 0.19047828]
 [0.27678239 0.12177714 0.19382821 ... 0.23307581 0.0673617  0.26891492]
 ...
 [0.58903166 0.2591588  0.41249356 ... 0.49601794 0.14335512 0.57228856]
 [0.42633888 0.18757815 0.29856128 ... 0.35901591 0.10375989 0.41422029]
 [0.04392812 0.01932724 0.03076246 ... 0.03699145 0.01069097 0.04267947]]
RMSE_missing: 1.9990277827520853e-05


#### Discussion the selection of lamda on the reconstruction accuracy

In [ ]:
lamda_all= [1e-6* i for i in range(0,11)]

for lamda in lamda_all:
    
    U_factorized,V_factorized = SGD_factorization(0.01, lamda, 1000, R_missing,False)

    R_reconstructed_missing = U_factorized @ V_factorized.T

    RMSE_missing = np.sqrt(np.mean((R_1 - R_reconstructed_missing)**2))

    print("******* Regularization:", lamda ,"********")
    print("RMSE of recovered matrix:", RMSE_missing)

    

******* Regularization: 0.0 ********
RMSE_missing: 1.7295401085124914e-05
******* Regularization: 1e-06 ********
RMSE_missing: 3.970605958899954e-05
******* Regularization: 2e-06 ********
RMSE_missing: 1.1413570757640905e-05
******* Regularization: 3e-06 ********
RMSE_missing: 3.053911236677769e-05
******* Regularization: 4e-06 ********
RMSE_missing: 1.0385992449091427e-05
******* Regularization: 4.9999999999999996e-06 ********
RMSE_missing: 1.084148953503658e-05
******* Regularization: 6e-06 ********
RMSE_missing: 4.53837481536864e-05
******* Regularization: 7e-06 ********
RMSE_missing: 3.914196535243085e-05
******* Regularization: 8e-06 ********
RMSE_missing: 3.8189555074555896e-05
******* Regularization: 9e-06 ********
RMSE_missing: 1.6332794224055282e-05
******* Regularization: 9.999999999999999e-06 ********
RMSE_missing: 6.885195566661244e-05


**Discussion**:  
I select lambda from 1e-6 to 1e-5.  
1. When the lambda is small, the regularization term has minimal effect, allowing U and V to fit the observed entries of R more closely. This increases the risk of overfitting. When the lambda is below 4e-6, the effect of reconstruction with regularization is similar to the effect of no regularization reconstruction.  
2. When the lambda is moderate such as 4e-06 and 5e-06, the regularization term could balance the generalization ability of learning to the observed entries and fittiing performance on the observed entries. So the RMSE is smaller than RMSE without regularization.  
3. When the lambda is large, the generalization ability arises but the regularization term dominates the objective function, causing the values in U and V to be smaller in magnitude. So the factorization suffers a underfitting.

#### Calculate and compare the RMSE

In [ ]:
# Calculate the RMSE of the recovere matrix

U_factorized,V_factorized = SGD_factorization(0.01,1e-5, 1000, R_missing,False)

R_reconstructed_missing = U_factorized @ V_factorized.T

RMSE_missing = np.sqrt(np.mean((R_1 - R_reconstructed_missing)**2))

print("RMSE of the recovered matrix:", RMSE_missing)

print("RMSE of the U", np.sqrt(np.mean((u - U_factorized)**2)))

print("RMSE of the V", np.sqrt(np.mean((v - V_factorized)**2)))

NameError: name 'SGD_factorization' is not defined